In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

#X_init = ... # 20x5 array
#y_init = ... # 20 outputs
X_init = datain
y_init = dataout


# Week 1

In [ ]:
# --- Step 1: Transform outputs for maximization ---
y_transformed = -y_init
y_best = y_transformed.max()

# --- Step 2: Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_transformed)

In [ ]:
# --- Step 3: Define Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because we will minimize

In [ ]:

# --- Step 4: Optimize acquisition function ---
bounds = [(0,1)]*5  # 5 ingredients
best_x = None
best_ei = float('inf')

# Multiple random starts to avoid local maxima
for _ in range(20):
    x0 = np.random.rand(5)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next recipe to try:", x_next)


# Week 2

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Step 0: Load and update data ---
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# Add the most recent data point
x_new = np.array([[0.861642, 0.308166, 0.510818, 0.325615, 0.845503]])
y_new = np.array([-1.7791349472320002])

# Combine with existing data
X_init = np.vstack([datain, x_new])
y_init = np.hstack([dataout, y_new])

print(X_init)
print(y_init)

In [ ]:
# --- Step 1: Transform outputs for maximization ---
y_transformed = -y_init   # since we want to make score close to zero (maximise negative of total)
y_best = y_transformed.max()

# --- Step 2: Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_transformed)

# --- Step 3: Define acquisition functions ---

# Expected Improvement (EI)
def expected_improvement(x, gp, y_best, xi=0.05):  # adjust xi for exploration/exploitation balance
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # we minimize in scipy

# --- (Optional) UCB version: uncomment to use ---
# def ucb(x, gp, kappa=2.0):
#     x = np.array(x).reshape(1, -1)
#     mu, sigma = gp.predict(x, return_std=True)
#     return -(mu + kappa * sigma)  # negative since we minimize

# --- Step 4: Optimize acquisition function ---
bounds = [(0, 1)] * 5
best_x = None
best_val = float('inf')

for _ in range(40):  # more restarts to reduce local optima risk
    x0 = np.random.rand(5)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    # --- For UCB, replace expected_improvement(...) with ucb(...)
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

x_next = best_x
print("Next recipe to try:", x_next)


# Week 3

Summary of changes:
✅ Added the two new data points (x_new1, x_new2)
✅ Reduced xi from 0.02 to 0.01 → focuses on exploitation
✅ Increased random restarts to 50 for robustness
💬 Kept UCB commented out (can easily be enabled later)

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Step 0: Load and update data ---
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# Add previously tested new points
x_new1 = np.array([[0.861642, 0.308166, 0.510818, 0.325615, 0.845503]])
y_new1 = np.array([-1.7791349472320002])

x_new2 = np.array([[0.52685018, 0.5778152, 0.74790401, 0.62958138, 0.84857269]])
y_new2 = np.array([-1.3401340011620242])

# Combine with existing data
X_init = np.vstack([datain, x_new1, x_new2])
y_init = np.hstack([dataout, y_new1, y_new2])

# --- Step 1: Transform outputs for maximization ---
y_transformed = -y_init
y_best = y_transformed.max()

# --- Step 2: Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_transformed)

# --- Step 3: Define acquisition functions ---

# Expected Improvement (EI)
def expected_improvement(x, gp, y_best, xi=0.01):  # reduced xi for more exploitation
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize because scipy.optimize minimizes by default

# --- (Optional) UCB version: uncomment to use ---
# def ucb(x, gp, kappa=2.0):
#     x = np.array(x).reshape(1, -1)
#     mu, sigma = gp.predict(x, return_std=True)
#     return -(mu + kappa * sigma)  # negative since we minimize

# --- Step 4: Optimize acquisition function ---
bounds = [(0, 1)] * 5
best_x = None
best_val = float('inf')

for _ in range(50):  # increased restarts for better global search
    x0 = np.random.rand(5)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    # --- For UCB, replace expected_improvement(...) with ucb(...)
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

x_next = best_x
print("Next recipe to try:", x_next)



In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Next recipe to try:", x_next_6dp)


# Week 4

The new point [0.304171, 0.964306, 0.527203, 0.006037, 0.376271] gave -1.8371,
which is worse than the previous new samples and far worse than the current best (-0.7143).
That tells us this region of input space is likely a poor area — we should record it and let the GP learn low values there.

Add the new point to the dataset and refit the GP.
Use Expected Improvement (EI) as the acquisition function but reduce xi to favor exploitation (I recommend xi = 0.005–0.01).
Compute one exploratory candidate using UCB with a larger kappa like 3.0) so you can occasionally sample high-uncertainty regions.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load original data ---
X_init = np.load('initial_inputs.npy')         # shape (20,5)
y_init = np.load('initial_outputs.npy')        # shape (20,)

# --- Append the previously tested new points (including the most recent) ---
x_new1 = np.array([0.861642, 0.308166, 0.510818, 0.325615, 0.845503])
y_new1 = -1.7791349472320002

x_new2 = np.array([0.52685018, 0.5778152, 0.74790401, 0.62958138, 0.84857269])
y_new2 = -1.3401340011620242

x_new3 = np.array([0.304171, 0.964306, 0.527203, 0.006037, 0.376271])   # most recent
y_new3 = -1.8370828066160314

X = np.vstack([X_init, x_new1, x_new2, x_new3])
y = np.hstack([y_init, y_new1, y_new2, y_new3])

# transform for maximization
y_trans = -y
y_best = y_trans.max()

# Fit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# Acquisition: Expected Improvement
def expected_improvement(x, gp, y_best, xi=0.005):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# Optional: Upper Confidence Bound (for exploration)
def ucb(x, gp, kappa=3.0):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + kappa * sigma)  # negative since we minimize

bounds = [(0.0, 1.0)] * X.shape[1]

# Helper optimizer that runs many restarts and also starts near a given seed
def optimize_acq(acq_func, gp, y_best, n_restarts=40, local_seeds=None):
    best_x = None
    best_val = np.inf
    # global random restarts
    for _ in range(n_restarts):
        x0 = np.random.rand(X.shape[1])
        res = minimize(lambda xx: acq_func(xx, gp, y_best),
                       x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.success and res.fun < best_val:
            best_val = res.fun
            best_x = res.x
    # local starts near seeds (if provided)
    if local_seeds is not None:
        for seed in local_seeds:
            for _ in range(8):  # a few noisy local starts
                x0 = seed + 0.05 * np.random.randn(X.shape[1])
                x0 = np.clip(x0, 0.0, 1.0)
                res = minimize(lambda xx: acq_func(xx, gp, y_best),
                               x0=x0, bounds=bounds, method='L-BFGS-B')
                if res.success and res.fun < best_val:
                    best_val = res.fun
                    best_x = res.x
    return best_x, best_val

# Identify current best input (from the dataset)
idx_best = np.argmax(y_trans)  # index of best transformed value
x_best = X[idx_best]

# Exploitative candidate: EI with local seeds near current best
x_next_exploit, val_e = optimize_acq(expected_improvement, gp, y_best,
                                     n_restarts=40, local_seeds=[x_best])

# Exploratory candidate: high-kappa UCB (optional)
x_next_explore, val_u = optimize_acq(lambda xx,gp,yb: ucb(xx,gp,kappa=3.0), gp, y_best,
                                     n_restarts=40, local_seeds=None)

print("Current best (transformed):", y_best, "at X =", x_best)
print("Exploitative next (EI, xi=0.005):", x_next_exploit)
print("Exploratory next (UCB, kappa=3.0):", x_next_explore)


# Going with Exploitative next (EI, xi=0.005): [0.13368735 0.61046861 0.48527119 0.33635839 0.75270271]


In [ ]:
x_next_6dp = np.round(x_next_exploit, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 5

- New input: [0.133687, 0.610469, 0.485271, 0.336358, 0.752703] → output -1.7633.
- That neighborhood already contains several poor values, so the new -1.76 is consistent with (and reinforces) that assessment.
- EI will be pushed toward regions with higher predicted means (the good basins) once the GP is updated.

- The new sample confirmed that part of the space is bad — don’t waste more samples there now.
- Focus on small, local steps around the current best to climb the nearby basin,
- but keep one periodic, deliberate exploration to escape local optima.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Load original data ---
X_init = np.load('initial_inputs.npy')         # shape (20,5)
y_init = np.load('initial_outputs.npy')        # shape (20,)

# --- Append the previously tested new points (including the most recent) ---
x_new1 = np.array([0.861642, 0.308166, 0.510818, 0.325615, 0.845503])
y_new1 = -1.7791349472320002

x_new2 = np.array([0.52685018, 0.5778152, 0.74790401, 0.62958138, 0.84857269])
y_new2 = -1.3401340011620242

x_new3 = np.array([0.304171, 0.964306, 0.527203, 0.006037, 0.376271]) 
y_new3 = -1.8370828066160314

x_new4 = np.array([0.133687, 0.610469, 0.485271, 0.336358, 0.752703])   # most recent
y_new4 = -1.7632847898105413

X = np.vstack([X_init, x_new1, x_new2, x_new3, x_new4])
y = np.hstack([y_init, y_new1, y_new2, y_new3, y_new4])


Computes one next point using Expected Improvement (EI) (exploit),
Computes one next point using Upper Confidence Bound (UCB) (explore).
- We are going to use the exploit version

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# === Transform outputs for maximisation ===
y_trans = -y        # since we want to bring score toward 0
y_best = np.max(y_trans)

# === Fit Gaussian Process ===
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X, y_trans)

# === Acquisition Functions ===
def expected_improvement(x, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0:
        return 0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization in scipy

def ucb(x, kappa=2.5):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + kappa * sigma)  # negative for minimization

# === Optimize acquisition function ===
bounds = [(0,1)] * 5

def optimize(acq):
    best_x = None
    best_val = float("inf")
    for _ in range(40):  # multiple random restarts
        x0 = np.random.rand(5)
        res = minimize(acq, x0=x0, bounds=bounds, method="L-BFGS-B")
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x
    return best_x

# === Next point to evaluate ===
x_next_EI = optimize(expected_improvement)
# x_next_UCB = optimize(lambda x: ucb(x))   # ← Uncomment to use exploration candidate

print("Next exploitative EI point:", x_next_EI)
# print("Next exploratory UCB point:", x_next_UCB)


In [ ]:
x_next_6dp = np.round(x_next_EI, 6)
print("Rounded to:", x_next_6dp)